# Carga Data AUDIENCIAS

Creación del DataFrame asociado a los audiencias

Rev: 29-10-2020

In [24]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [25]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [26]:
# Creacion dataframe con los datos de Audiencias Realizadas Penal de los años 2015 a 2019

dataframes = []

for archivo in archivos:
    if archivo.find("Audiencias Realizadas Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', dtype = 'unicode', low_memory = True)
        dataframes.append(df) 
        
df_audiencias = pd.concat(dataframes, axis = 0)


In [27]:
df_audiencias.columns

Index(['CÓD. CORTE', 'CORTE', 'CÓD. TRIBUNAL', 'TRIBUNAL', 'RIT', 'TIPO CAUSA',
       'TIPO DE AUDIENCIA', 'FECHA PROGRAMACIÓN AUDIENCIA', 'FECHA AUDIENCIA',
       'AGENDAMIENTO (DÍAS CORRIDOS)', 'DURACIÓN AUDIENCIA (MINUTOS)',
       'TOTAL AUDIENCIAS', 'TIPO AUDIENCIA',
       'PLAZO AGENDAMIENTO (DIAS CORRIDOS)', 'DURACIÓN  AUDIENCIA'],
      dtype='object')

In [28]:
df_audiencias.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                'DURACIÓN  AUDIENCIA':'DURACION AUDIENCIA',
                                'AGENDAMIENTO (DÍAS CORRIDOS)':'DIAS AGENDAMIENTO',
                                'DURACIÓN AUDIENCIA (MINUTOS)':'DURACION AUDIENCIA (MIN)',
                                'FECHA PROGRAMACIÓN AUDIENCIA':'FECHA PROGRAMACION AUDIENCIA'
                               },
                     inplace = True)

# TRANSFORMAMOS DE FLOAT A INTEGER

df_audiencias['COD. CORTE'] = df_audiencias['COD. CORTE'].fillna(0).astype(np.int16)
df_audiencias['COD. TRIBUNAL'] = df_audiencias['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_audiencias['TOTAL AUDIENCIAS'] = df_audiencias['TOTAL AUDIENCIAS'].fillna(0).astype(np.int8)

In [29]:
# Podemos observar que existen columnas que se repiten, y que tienen datos NAN en algunas pero esos datos 
# en otras columnas, pasa en TIPO AUDIENCIA=TIPO DE AUDIENCIA, AGENDAMIENTO (DÍAS CORRIDOS)=PLAZO AGENDAMIENTO
# (DÍAS CORRIDOS), DURACIÓN AUDIENCIA (MINUTOS)= DURACIÓN AUDIENCIA

df_audiencias['TIPO DE AUDIENCIA'] = df_audiencias['TIPO DE AUDIENCIA'].fillna(df_audiencias['TIPO AUDIENCIA'])
df_audiencias['DIAS AGENDAMIENTO'] = df_audiencias['DIAS AGENDAMIENTO'].fillna(df_audiencias['PLAZO AGENDAMIENTO (DIAS CORRIDOS)']).astype(np.int16)
df_audiencias['DURACION AUDIENCIA (MIN)'] = df_audiencias['DURACION AUDIENCIA (MIN)'].fillna(df_audiencias['DURACION AUDIENCIA'])

In [30]:
# Elimino las columnas reemplazadas

df_audiencias.drop(['TIPO AUDIENCIA','PLAZO AGENDAMIENTO (DIAS CORRIDOS)','DURACION AUDIENCIA'], 
                   axis = 'columns', inplace = True)

In [31]:
# Transformamos fechas

df_audiencias['FECHA PROGRAMACION AUDIENCIA'] = df_audiencias['FECHA PROGRAMACION AUDIENCIA'].progress_apply(cleandata.convierte_fecha)
df_audiencias['FECHA AUDIENCIA'] = df_audiencias['FECHA AUDIENCIA'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 6712514/6712514 [00:12<00:00, 541318.29it/s]


In [32]:
# Elimino espacios en las columnas tipo objetos

df_audiencias = df_audiencias.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 12/12 [01:06<00:00,  5.56s/it]


In [33]:
# Elimino tildes 

cols = df_audiencias.select_dtypes(include = ["object"]).columns
df_audiencias[cols] = df_audiencias[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 6/6 [01:51<00:00, 18.61s/it]


In [34]:
# Categorizar

df_audiencias['CORTE'] = df_audiencias['CORTE'].astype('category')

Dejo solo causas Ordinarias

In [35]:
df_audiencias['TIPO CAUSA'].unique()

array(['Ordinaria', 'Exhorto', 'Extradición', 'Militar'], dtype=object)

In [36]:
tipo_causa = df_audiencias[df_audiencias['TIPO CAUSA']!='Ordinaria']

In [37]:
df_audiencias.drop(tipo_causa.index, axis=0, inplace=True)

In [38]:
# Reset el index para realizar feather

df_audiencias.reset_index(inplace = True)
df_audiencias.sort_values('FECHA PROGRAMACION AUDIENCIA')

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,TIPO DE AUDIENCIA,FECHA PROGRAMACION AUDIENCIA,FECHA AUDIENCIA,DIAS AGENDAMIENTO,DURACION AUDIENCIA (MIN),TOTAL AUDIENCIAS
5911937,1028429,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA DE PUERTO MONTT,624-2006,Ordinaria,AUDIENCIA DE COMUNI. NO PERSEV. PROCED.,2007-04-25,2015-10-23,3103,1,1
5909127,1025242,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA DE PUERTO MONTT,3818-2006,Ordinaria,AUDIENCIA DE MEDIDAS CAUTELARES.,2007-07-30,2015-10-15,2999,2,1
5906737,1022513,56,C.A. DE PUERTO MONTT,1090,JUZGADO DE GARANTIA DE PUERTO MONTT,1026-2005,Ordinaria,AUDIENCIA DE PROCEDIMIENTO ABREVIADO.,2008-02-14,2015-10-23,2808,5,1
5022886,57134,15,C.A. DE ANTOFAGASTA,952,JUZGADO DE GARANTIA DE ANTOFAGASTA,6143-2007,Ordinaria,AUDIENCIA DE PREPARACION JUICIO ORAL.,2008-03-31,2015-06-18,2635,5,1
625821,683790,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA DE ARICA,1804-2006,Ordinaria,AUDIENCIA DE SOBRESEIMIENTO.,2008-10-01,2018-08-24,3614,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747420,1334990,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA DE TALAGANTE,994-2016,Ordinaria,AUDIENCIA DE LEY 18.216.,NaT,2017-09-13,21,2,1
3747422,1334992,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA DE TALAGANTE,997-2017,Ordinaria,AUDIENCIA DE FORMAL. DE LA INVESTIGACISN.,NaT,2017-12-04,39,1,1
3747425,1334995,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA DE TALAGANTE,998-2017,Ordinaria,AUDIENCIA DE FORMAL. DE LA INVESTIGACISN.,NaT,2017-10-05,38,2,1
3747426,1334996,91,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA DE TALAGANTE,998-2017,Ordinaria,AUDIENCIA DE FORMAL. DE LA INVESTIGACISN.,NaT,2017-11-16,42,5,1


In [39]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_audiencias.to_feather(f'{path_interim}/clean_Audiencias.feather')